In [ ]:

# Ensure repository root (with 'Source/ai') is on sys.path
from pathlib import Path
import sys

project_root = next((p for p in [Path.cwd(), *Path.cwd().parents] if (p / 'Source' / 'ai').exists()), None)
if project_root and str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from dotenv import load_dotenv
from langchain_ollama import ChatOllama
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
from langchain.tools import Tool
from langgraph.graph import END, StateGraph
import operator
import requests
from Source.ai.Multi_Agent.Source.Main.Tools import poem_tools, sentiment_tools, travel_tools, weather_tools
from Source.ai.Multi_Agent.Source.Main.Agents.Agents_1 import Coordinator_Agent_1, Flight_Agent_1, Hotel_Agent_1, Travel_Agent_1
from Source.ai.Multi_Agent.Source.Main.Agents.Agents_2 import Coordinator_Agent_2, Flight_Agent_2, Hotel_Agent_2, Travel_Agent_2
from Source.ai.Multi_Agent.Source.Main.Agents.Agents_3 import Coordinator_Agent_3, Flight_Agent_3, Hotel_Agent_3, Travel_Agent_3, Aggregator_Agent_3
from Source.ai.Multi_Agent.Source.Main.Agents.Agents_Summary import Abstracter_Agent, Aggregator_Agent, Coordinator_Agent, Evaluator_Agent, Extractor_Agent, GradeCalibrator_Agent, OCR_Agent, SpellChecker_Agent
from typing import TypedDict, Annotated, List, Any, Dict, Literal
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from Source.ai.Multi_Agent.Source.Main.Memory.memory.memory import memory_manager
from Source.ai.Multi_Agent.Source.Main.Memory.memory.long_term_memory import long_term_memory
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import uuid
import os
from langchain_openai import ChatOpenAI

In [14]:
load_dotenv()

llm = ChatOllama(model="llama3:8b") # <-- Sử dụng model bạn đã kéo về, ví dụ "llama3", "mistral"

class AgentState(TypedDict):
    input: str
    messages: Annotated[List[str], operator.add]
    
prompt = hub.pull("hwchase17/react")

In [15]:
def new_session(user_id: str = "default_user", clear_history: bool = True, keep_preferences: bool = True, auto_continue: bool = False, replay_last_n: int = 20) -> str:
    sid = memory_manager.start_new_session(user_id=user_id, clear_history=clear_history, keep_preferences=keep_preferences)
    print(f"New session started: {sid}")
    if auto_continue:
        initial_state = build_state_from_memory(user_id=user_id, max_messages=replay_last_n)
        run_langgraph_chat(initial_state=initial_state)
    return sid

sid = new_session()

New session started: session_20251027_170231


In [16]:
all_items = long_term_memory.collection.get(include=["metadatas"])
session_ids = sorted({m.get("session_id") for m in all_items["metadatas"] if m})
print(session_ids)

['q', 'session_20250925_130323', 'session_20250925_144950', 'session_20250925_150303', 'session_20250925_151001', 'session_20250927_162902', 'session_20250927_164507', 'session_20250928_092221', 'session_20250928_154914', 'session_20250928_155638', 'session_20251001_160456', 'session_20251002_093329', 'session_20251002_110113', 'session_20251002_143156']


In [17]:
def create_initial_state() -> AgentState:
    return {
        "messages": [],
        "current_agent": "coordinator",
        "needs_user_input": False,
        "conversation_stage": "greeting",
    }
state = create_initial_state()
try:
    state = app.invoke(state, config={"recursion_limit": 20})
except Exception:
    pass

print("Nodes wired: aggregator added between specialist agents and coordinator.")


Nodes wired: aggregator added between specialist agents and coordinator.


In [18]:
def read_long_term_memory_by_session_id(session_id: str):
    col = long_term_memory.collection
    all_items = col.get(include=["documents","metadatas"])
    for doc, meta in zip(all_items["documents"], all_items["metadatas"]):
        if meta.get("session_id") == session_id:
            print(meta.get("timestamp"), meta.get("session_id"), meta.get("role"), ":", doc)

In [ ]:
# Khai báo State cho LangGraph
class AgentState(TypedDict):
    messages: List[Any]
    current_agent: str
    needs_user_input: bool
    conversation_stage: Literal["greeting", "reader_ocr", "spellchecker", "extractor", "abstracter", "grade_calibrator", "evaluator", "aggregator", "completed"]
    check: Literal["exactly", "not exactly"] = Field(
        description="Decide if the text is exactly the same as the original text.",
    )
    feedback: str = Field(
        description="If the text is not exactly the same as the original text, provide feedback on how to improve it.",
    )

# Hàm điều hướng sau node coordinator
def decide_next_agent(state: AgentState):
    if state.get("needs_user_input", False):
        return "END"
    return state.get("current_agent", "coordinator_agent")

def route_check(state: AgentState):
    if state["check"] == "exactly":
        return "Accepted"
    elif state["check"] == "not exactly":
        return "Rejected + Feedback"
    
# Build đồ thị LangGraph
workflow = StateGraph(AgentState)

workflow.add_node("coordinator_agent", Coordinator_Agent.coordinator_agent)
workflow.add_node("reader_ocr_agent", OCR_Agent.ocr_agent)
workflow.add_node("spellchecker_agent", SpellChecker_Agent.spellchecker_agent)
workflow.add_node("extractor_agent", Extractor_Agent.extractor_agent)
workflow.add_node("abstracter_agent", Abstracter_Agent.abstracter_agent)
workflow.add_node("grade_calibrator_agent", GradeCalibrator_Agent.grade_calibrator_agent)
workflow.add_node("evaluator_agent", Evaluator_Agent.evaluator_agent)
workflow.add_node("aggregator_agent", Aggregator_Agent.aggregator_agent)

workflow.set_entry_point("coordinator_agent")

workflow.add_conditional_edges(
    "coordinator_agent",
    decide_next_agent,
    {
        "reader_ocr_agent": "reader_ocr_agent",
        "spellchecker_agent": "spellchecker_agent",
        "extractor_agent": "extractor_agent",
        "abstracter_agent": "abstracter_agent",
        "grade_calibrator_agent": "grade_calibrator_agent",
        "evaluator_agent": "evaluator_agent",
        "aggregator_agent": "aggregator_agent",
        "coordinator_agent": "coordinator_agent",
        "END": END,
    },
)
workflow.add_edge("reader_ocr_agent", "spellchecker_agent")
workflow.add_conditional_edges(
    "spellchecker_agent",
    route_check,
    {  
        "Accepted": END,
        "Rejected + Feedback": "reader_ocr_agent",
    },
)
workflow.add_edge("spellchecker_agent", "extractor_agent")
workflow.add_edge("spellchecker_agent", "abstracter_agent")
workflow.add_edge("extractor_agent", "grade_calibrator_agent")
workflow.add_edge("abstracter_agent", "grade_calibrator_agent")
workflow.add_edge("grade_calibrator_agent", "evaluator_agent")
workflow.add_edge("evaluator_agent", "aggregator_agent")
workflow.add_edge("aggregator_agent", END)

app = workflow.compile()

def run_langgraph_chat(initial_state=None):
    print("🤖 Multi-Agent System Summary For Primary School Students")
    print("=" * 60)
    print("Commands: 'exit', 'clear' (STM), 'clear_all' (STM+LTM), 'mem_stats'")

    state = initial_state or create_initial_state()

    # KHÔNG auto-invoke nếu đã có messages (tránh chào lại)
    if not state.get("messages"):
        try:
            state = app.invoke(state, config={"recursion_limit": 50})
            last = state["messages"][-1] if state["messages"] else None
            if last and isinstance(last, AIMessage):
                print(f"\n🤖{state['current_agent']}: {last.content}")
        except Exception:
            pass

    while True:
        if not state.get("needs_user_input", True):
            state = app.invoke(state, config={"recursion_limit": 50})
            last = state["messages"][-1] if state["messages"] else None
            if last and isinstance(last, AIMessage):
                print(f"\n🤖{state['current_agent']}: {last.content}")
            mem = memory_manager.get_memory()
            print(f"   [Memory: {len(mem.conversation_history)} msgs, {len(mem.user_preferences)} prefs]")
            continue

        user_input = input("\n👤 Bạn: ").strip()
        memory_manager.add_message("user", user_input)

        if user_input.lower() in ["exit", "quit", "thoát"]:
            print("👋 Bye MAS Lịch sử chat đã được lưu.")
            break
        if user_input.lower() in ["clear", "xóa", "reset"]:
            memory_manager.clear_memory()
            state = create_initial_state()
            print("🧹 Đã xóa short-term memory. Long-term vẫn giữ.")
            continue
        if user_input.lower() in ["clear_all", "xóa_all", "reset_all"]:
            memory_manager.clear_memory(also_long_term=True)
            state = create_initial_state()
            print("🧹 Đã xóa cả short-term và long-term memory.")
            continue
        if user_input.lower() in ["mem_stats", "memory_stats"]:
            print(f"📊 Long-term Memory: {long_term_memory.collection.count()} items")
            continue

        state["messages"].append(HumanMessage(content=user_input))
        print(f"👤: {user_input}")
        state["needs_user_input"] = False
def build_state_from_memory(user_id: str = "default_user", max_messages: int = 10):
    mem = memory_manager.get_memory(user_id)
    msgs = []
    ctrl = {"thoát","exit","quit","xóa","clear","reset","clear_all","xóa_all","reset_all"}
    for m in mem.conversation_history[-max_messages:]:
        content = (m.get("content") or "").strip()
        if content.lower() in ctrl:
            continue
        role = (m.get("role") or "").lower()
        if role == "user":
            msgs.append(HumanMessage(content=content))
        else:
            msgs.append(AIMessage(content=content))
    needs_user_input = True if msgs and isinstance(msgs[-1], AIMessage) else False
    return {
        "messages": msgs,
        "current_agent": "coordinator_agent",
        "needs_user_input": needs_user_input,
        "conversation_stage": "reader_ocr",
    }
def continue_chat_from_session(session_id: str, user_id: str = "default_user", replay_last_n: int = 20):
    print("Previous chat history:")
    read_long_term_memory_by_session_id("session_20251027_170231")
    loaded = memory_manager.resume_session(session_id, user_id=user_id, replay_last_n=replay_last_n)
    print(f"Resumed {loaded} messages from long-term: {session_id}")
    initial_state = build_state_from_memory(user_id=user_id, max_messages=replay_last_n)
    run_langgraph_chat(initial_state=initial_state)

# run_langgraph_chat()
# hoặc tiếp nối từ 1 session cụ thể:
continue_chat_from_session("session_20251027_170231")

Previous chat history:
2025-09-25T15:10:17.893726 session_20250925_151001 assistant : assistant: Xin chào! Tôi là trợ lý du lịch đa nhiệm. Tôi có thể giúp gì cho chuyến đi của bạn?
2025-09-25T15:10:21.472032 session_20250925_151001 user : user: thoát
2025-09-25T15:15:57.619270 session_20250925_151001 user : user: thoát
2025-09-25T15:19:04.791208 session_20250925_151001 user : user: thoát
2025-09-25T15:20:39.525862 session_20250925_151001 user : user: Tôi muốn lên kế hoạch du lịch hà Nội
2025-09-25T15:20:39.938372 session_20250925_151001 user : user: Tôi muốn lên kế hoạch du lịch hà Nội
2025-09-25T15:21:11.216768 session_20250925_151001 assistant : assistant: Hà Nội - một trong những thành phố cổ kính và giàu có về di tích lịch sử của Việt Nam.

Để giúp tôi phục vụ bạn tốt hơn, có thể chia sẻ với tôi về ý định du lịch của bạn. Bạn muốn đi đâu? Bạn có nhu cầu gì (ví dụ: nghỉ dưỡng, thăm quan di tích, trải nghiệm văn hóa)? 

Tôi sẽ giúp bạn lên kế hoạch du lịch hà Nội phù hợp nhất!
2025-0

In [19]:
read_long_term_memory_by_session_id("session_20251027_170231")